The dataset for this BINARY CLASSIFIER to be used is [bc-dataset.csv].
There are 2 implementations here:
1. Bag of words approach.
2. Word vectors (can be pre-trained word embeddings). 

The dataset split is <>.
Evaluation metrics to be used in this are:
1. Precision.
2. Recall.
3. F-Measure.

Loads the dataset.

In [53]:
from ftfy import fix_encoding
import pandas as pd
import numpy as np
import re
import csv
import nltk

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

def fix_encode(x):
    return fix_encoding(x)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv(
    "datasets/bc-dataset.csv",
    encoding="latin-1",
    sep=",",
    quoting=csv.QUOTE_ALL
)

data = pd.concat([df.gender, df.description], axis=1)

#drop null rows
data.dropna(axis=0, inplace=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Shows some descriptive stats (just for fun).
Also applies ftfy encoding to the ['text'] Column to fix any broken encodings when the .csv file was loaded.

In [54]:
data.description = data.description.apply(lambda x: fix_encode(x))

In [ ]:
# print(df['text'].head(10))##
# print(df['text'].tail(10))##

# print(data.head(15))

In [58]:
print("Shape: " + str(data.shape))

print("Just some stats.")
print("------")
print(data["gender"].describe())
print("------")
print(data["gender"].value_counts(ascending=True))

Shape: (16224, 2)
Just some stats.
------
count      16224
unique         4
top       female
freq        5725
Name: gender, dtype: object
------
unknown     702
brand      4328
male       5469
female     5725
Name: gender, dtype: int64


Change gender strings to integer values.